In [4]:
import findspark

findspark.init('/opt/spark')

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lec13_df_moynihanl').getOrCreate()

In [6]:
url = 'https://raw.githubusercontent.com/BlueJayADAL/DS420/master/datasets/dataframe/people.json'

In [7]:
spark.sparkContext.addFile(url)

In [8]:
from pyspark import SparkFiles

In [9]:
fileloc = SparkFiles.get('people.json')
fileloc

'/tmp/spark-e1382fd8-a6ec-45f5-bb25-e589837e800a/userFiles-c17a5ed7-f273-4c54-88ef-c823ab70e9ea/people.json'

## Section 1: Spark dataframe Basics

In [10]:
df = spark.read.json('file://'+fileloc)

In [11]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [12]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [13]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [14]:
df.columns

['age', 'name']

In [15]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [16]:
from pyspark.sql.types import (StructField, StructType, StringType, IntegerType)

In [17]:
data_schema = [StructField('age', IntegerType(), True), 
              StructField('name', StringType(), True)]

In [18]:
final_struct = StructType(data_schema)

In [19]:
df = spark.read.json('file://'+fileloc, schema = final_struct)

In [20]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [21]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [22]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [23]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [24]:
df = df.withColumn('double_age', df['age']*2)

In [25]:
df.show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [26]:
df.withColumnRenamed('double_age','oldage').show()

+----+-------+------+
| age|   name|oldage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [27]:
df.createOrReplaceTempView('my_people')

In [28]:
sql_result = spark.sql('select * from my_people where age = 30')

In [29]:
sql_result.show()

+---+----+----------+
|age|name|double_age|
+---+----+----------+
| 30|Andy|        60|
+---+----+----------+



## Dataframe Operations

In [30]:
url = 'https://raw.githubusercontent.com/BlueJayADAL/DS420/master/datasets/dataframe/appl_stock.csv'

spark.sparkContext.addFile(url)

stocks = spark.read.csv(path='file://'+SparkFiles.get('appl_stock.csv'),
                                inferSchema = True,
                                header=True)

In [31]:
stocks.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [32]:
stocks.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [33]:
##boolen selection

In [34]:
stocks[stocks['Close'] < 500].show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [35]:
stocks.filter(stocks['Close'] < 500).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [36]:
stocks[(stocks['Close'] < 200) & (stocks['Open'] > 200)].show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [37]:
stocks.filter((stocks['Close']<200) & (stocks['Open'] > 200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [38]:
result = stocks.filter(stocks['Low'] == 197.16).collect()
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [39]:
type(result)

list

In [40]:
result[0].asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [41]:
for key, value in result[0].asDict().items():
    print(key, value)

Date 2010-01-22
Open 206.78000600000001
High 207.499996
Low 197.16
Close 197.75
Volume 220441900
Adj Close 25.620401


## Groupby and Aggregation

In [42]:
url = 'https://raw.githubusercontent.com/BlueJayADAL/DS420/master/datasets/dataframe/sales_info.csv'

In [43]:
spark.sparkContext.addFile(url)

In [46]:
sales = spark.read.csv('file://'+SparkFiles.get('sales_info.csv'),
                      inferSchema = True, 
                      header = True)

In [47]:
sales.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [53]:
sales.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [54]:
sales.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [55]:
sales.agg({'Sales':'max'}).show() #when u dont care about groups use agg

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [56]:
sales.groupBy('Company').agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [57]:
import pyspark.sql.functions as F

In [61]:
sales.select(F.countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [63]:
sales.select(F.avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [64]:
sales_std = sales.select(F.stddev('Sales').alias('std'))

sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [67]:
sales_std.select(F.format_number('std', 2).alias('std w/ 2 decimal places')).show()

+-----------------------+
|std w/ 2 decimal places|
+-----------------------+
|                 250.09|
+-----------------------+



In [68]:
sales.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [69]:
sales.orderBy(F.desc('Sales')).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [ ]:
sales.orderBy(F.desc('Sales')).show()